In [51]:
# 저자 깃허브 url
path = 'https://raw.githubusercontent.com/wesm/pydata-book/refs/heads/3rd-edition'

# Chapter 7. 데이터 정제 및 준비

## 7.1 누락된(결측) 데이터 처리하기
- 판다스의 설계 목표 중 하나는 누락된 데이터를 가능한 한 쉽게 처리하는 것
- 판다스 객체의 모든 기술 통계는 기본적으로 누락된 데이터를 배제하고 처리
- 감싯값(sentinel value): 누락되거나 null 값임을 나타내기 위해 등장

<br>[결측값]
- NaN(Not a Number): float64 데이터 타입에서 결측값을 나타내는 데 사용되는 sentinel 값
- NA(Not Available): 데이터가 존재하지 않거나 관찰되지 않았음을 의미

<br>[NA 처리 메서드]
- dropna: 누락된 데이터가 있는 축 제외. 어느 정도의 누락 데이터까지 용인할 것인지 지정 가능
- fillna: 누락된 데이터를 대신할 값을 채우거나 ffill이나 bfill같은 보간 메서드 적용
- isna: 누락되거나 NA인 값을 알려주는 불리언 값 반환
- notna: isna와 반대되는 메서드. NA가 아닌 값에는 True, NA값에는 False

In [52]:
import pandas as pd
import numpy as np

In [53]:
float_data = pd.Series([1.2, -3.5, np.nan, 0])
float_data

,0
0,1.2
1,-3.5
2,NaN
3,0.0


In [54]:
# isna: 값이 null인 경우 True
float_data.isna()

,0
0,False
1,False
2,True
3,False


In [55]:
print(float_data)
print(float_data.isna())

0    1.2
1   -3.5
2    NaN
3    0.0
dtype: float64
0    False
1    False
2     True
3    False
dtype: bool


In [56]:
string_data = pd.Series(["aardvark", np.nan, None, "avocado"])
string_data

,0
0,aardvark
1,NaN
2,None
3,avocado


In [57]:
string_data.isna() # None 또한 NA 값으로 취급

,0
0,False
1,True
2,True
3,False


In [58]:
float_data = pd.Series([1, 2, None], dtype=np.float64)
float_data

,0
0,1.0
1,2.0
2,NaN


In [59]:
float_data.isna()

,0
0,False
1,False
2,True


##7.1.1 누락된 데이터 골라내기
- Series에 dropna 메서드를 적용하면 널이 아닌 데이터와 색인값만 들어 있는 Series 반환
- DataFrame 객체의 경우 모두 NA인 행이나 열을 제외하거나 NA 값을 하나라도 포함하고 있는 행이나 열 제외
> dropna의 경우 기본적으로 NA 값이 하나라도 있는 행 제외 (원본 객체의 내용을 변경하지 않고 새로운 객체 반환)
<br> how="all" 옵션은 모든 값이 NA인 행만 제외 / how="any" : 하나라도 NaN 있으면 삭제
<br> thresh : 결측치가 특정 개수보다 적은 행만 살펴보고 싶을 경우

|방법|    설명|
---- |-------
dropna|    각 레이블의 값에 누락된 데이터가 있는지 여부에 따라 축 레이블을 필터링하며, 허용할 누락 데이터 양에 대한 임계값을 다양하게 설정.
fillna|    누락된 데이터를 일부 값으로 채우거나 "ffill" 또는 "bfill"과 같은 보간법을 사용 .
isna|    누락된 값/NA 값을 나타내는 부울 값을 반환.
notna|    isna와 반대. NA가 아닌 값에 True, NA 값에는 False

In [60]:
float_data.dropna()

,0
0,1.0
1,2.0


In [61]:
float_data[float_data.notna()]

,0
0,1.0
1,2.0


In [62]:
data = pd.Series([1, np.nan, 3.5, np.nan, 7])
data

,0
0,1.0
1,NaN
2,3.5
3,NaN
4,7.0


In [63]:
data.dropna()

,0
0,1.0
2,3.5
4,7.0


In [64]:
data[data.notna()]

,0
0,1.0
2,3.5
4,7.0


In [65]:
# DataFrame에서 dropna의 경우 기본적으로 NA 값이 하나라도 있는 행 제외
data = pd.DataFrame([[1., 6.5, 3.], [1., np.nan, np.nan],
                     [np.nan, np.nan, np.nan], [np.nan, 6.5, 3.]])
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [66]:
# n이 하나라도 있으면 행 삭제
data.dropna()

,0,1,2
0,1.0,6.5,3.0


In [67]:
# how="all" 옵션은 모든 값이 NA인 행만 제외
data.dropna(how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [68]:
# DF에서 열 제외
data[3] = np.nan # 모두 NaN인 열 추가
data

,0,1,2,3
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [69]:
data.dropna(axis=1, how="all") # column = 1

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [70]:
data.dropna(axis="columns", how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [71]:
data.dropna(axis=0, how="all") # index=0

,0,1,2,3
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [72]:
data.dropna(axis="index", how="all")

,0,1,2,3
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [73]:
# thresh : 결측치가 특정 개수보다 적은 행만 살펴보고 싶을 경우
df = pd.DataFrame(np.random.standard_normal((7,3)))
df

,0,1,2
0,0.595158,-1.918411,-2.219739
1,-0.554051,-1.340875,1.421678
2,-0.292515,0.326238,-1.030644
3,-0.096448,0.392746,-0.304878
4,0.136668,0.544819,0.484889
5,1.142108,-0.440866,1.033012
6,-0.262500,-0.018900,-1.946411


In [74]:
df.iloc[:4, 1] = np.nan # 0~3행, 1열
df.iloc[:2, 2] = np.nan # 0~1행, 2열
df

,0,1,2
0,0.595158,NaN,NaN
1,-0.554051,NaN,NaN
2,-0.292515,NaN,-1.030644
3,-0.096448,NaN,-0.304878
4,0.136668,0.544819,0.484889
5,1.142108,-0.440866,1.033012
6,-0.262500,-0.018900,-1.946411


In [75]:
df.dropna()

,0,1,2
4,0.136668,0.544819,0.484889
5,1.142108,-0.440866,1.033012
6,-0.262500,-0.018900,-1.946411


In [76]:
# thresh 매개변수는 특정 행 또는 열에 유지되기 위해 필요한 최소한의 비결측값 개수 지정
df.dropna(thresh=2) # 결측치가 2개보다 적은 행만 출력

,0,1,2
2,-0.292515,NaN,-1.030644
3,-0.096448,NaN,-0.304878
4,0.136668,0.544819,0.484889
5,1.142108,-0.440866,1.033012
6,-0.262500,-0.018900,-1.946411


### 7.1.2 결측치 채우기
- 데이터 상의 빈 곳을 어떻게든 매우고 싶다면 fillna 메서드 사용

[fillna 함수 인수]
- value: 비어 있는 값을 채울 스칼라 값이나 딕셔너리 형식의 객체
- method: 보간 방식이며 bfill: 뒤로 채우기 / ffill: 앞으로 채우기 (기본값은 None)
- axis: 값을 채워 넣을 축 (index - 0 또는 columns - 1). 기본값은 axis="index"
- limit: 값을 앞 혹은 뒤에서부터 몇 개까지 채울지 지정
-  평균값(mean)이나 중간값(median) 같은 통곗값을 채울 수도 있다 > fillna(data.mean())

In [77]:
df.fillna(0)

,0,1,2
0,0.595158,0.000000,0.000000
1,-0.554051,0.000000,0.000000
2,-0.292515,0.000000,-1.030644
3,-0.096448,0.000000,-0.304878
4,0.136668,0.544819,0.484889
5,1.142108,-0.440866,1.033012
6,-0.262500,-0.018900,-1.946411


In [78]:
# fillna에 딕셔너리 값을 넘기면 각 열마다 다른 값이 채워짐
df.fillna({1: 0.5, 2:0})

,0,1,2
0,0.595158,0.500000,0.000000
1,-0.554051,0.500000,0.000000
2,-0.292515,0.500000,-1.030644
3,-0.096448,0.500000,-0.304878
4,0.136668,0.544819,0.484889
5,1.142108,-0.440866,1.033012
6,-0.262500,-0.018900,-1.946411


In [79]:
# 보간메서드
df = pd.DataFrame(np.random.standard_normal((6, 3)))
df

,0,1,2
0,-1.899032,1.347636,0.363280
1,-0.420523,-0.920133,-0.013235
2,1.058527,-0.108553,-0.238404
3,-1.454474,-1.197828,0.973092
4,-0.975386,1.706488,0.460901
5,0.010096,0.742457,0.520664


In [80]:
df.iloc[2:, 1] = np.nan # 2~마지막 행, 1열
df.iloc[4:, 2] = np.nan # 4~마지막 행, 2열
df

,0,1,2
0,-1.899032,1.347636,0.363280
1,-0.420523,-0.920133,-0.013235
2,1.058527,NaN,-0.238404
3,-1.454474,NaN,0.973092
4,-0.975386,NaN,NaN
5,0.010096,NaN,NaN


In [81]:
df.fillna(method="ffill")

<ipython-input-81-5a0d178a1f3e>:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill")


,0,1,2
0,-1.899032,1.347636,0.363280
1,-0.420523,-0.920133,-0.013235
2,1.058527,-0.920133,-0.238404
3,-1.454474,-0.920133,0.973092
4,-0.975386,-0.920133,0.973092
5,0.010096,-0.920133,0.973092


In [82]:
df.fillna(method="ffill", limit=2) # 앞의 값을 다 채우되, 2개만 채움

<ipython-input-82-14ff2c1859d1>:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill", limit=2) # 앞의 값을 다 채우되, 2개만 채움


,0,1,2
0,-1.899032,1.347636,0.363280
1,-0.420523,-0.920133,-0.013235
2,1.058527,-0.920133,-0.238404
3,-1.454474,-0.920133,0.973092
4,-0.975386,NaN,0.973092
5,0.010096,NaN,0.973092


In [83]:
# 평균값이나 중간값을 채울 수도 있다
data = pd.Series([1., np.nan, 3.5, np.nan, 7])
data

,0
0,1.0
1,NaN
2,3.5
3,NaN
4,7.0


In [84]:
data.fillna(data.mean())

,0
0,1.000000
1,3.833333
2,3.500000
3,3.833333
4,7.000000


In [85]:
data.fillna(data.median())

,0
0,1.0
1,3.5
2,3.5
3,3.5
4,7.0


## 7.2. 데이터 변형
- 필터링
- 정제
- 변형

### 7.2.1 중복 제거하기
-  duplicated 메서드: 각 행의 중복 여부 불리언 값으로 반환
- drop_duplicates: duplicated 배열이 False인 DataFrame 필터링하여 반환
- duplicated와 drop_duplicated는 기본적으로 처음 발견된 값 유지
> keep="last" 옵션은 마지막으로 발견된 값 반환

In [86]:
data = pd.DataFrame({"k1": ["one", "two"] * 3 + ["two"],
                     "k2": [1, 1, 2, 3, 3, 4, 4]
                     })
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [87]:
# duplicated
data.duplicated()

,0
0,False
1,False
2,False
3,False
4,False
5,False
6,True


In [88]:
# drop_duplicates: duplicated 배열이 False인 DataFrame 필터링하여 반환
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [89]:
# 새로운 열 추가
data["v1"] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [90]:
# k1열에 기반해서 중복 걸러내기
data.drop_duplicates(subset=["k1"])

,k1,k2,v1
0,one,1,0
1,two,1,1


In [91]:
# duplicated와 drop_duplicated는 기본적으로 처음 발견된 값 유지
# keep="last" 옵션은 마지막으로 발견된 값 반환
data.drop_duplicates(subset=["k1", "k2"], keep="last")

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### 7.2.2 함수나 매핑을 이용해서 데이터 변형
- Series의 map 메서드: 변형을 위한 매핑 정보가 담긴 딕셔너리 같은 객체나 함수를 인수로 받음

In [92]:
# 가상으로 수집한 육류 관련 정보
data = pd.DataFrame({"food": ["bacon", "pulled pork", "bacon",
                              "pastrami", "corned beef", "bacon",
                              "pastrami", "honey ham", "nova lox"],
                     "ounces": [4,3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,pastrami,6.0
4,corned beef,7.5
5,bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [93]:
# 해당 육류의 원재료 알려주는 열 추가
meat_to_animal = {
    "bacon": "pig",
    "pulled pork": "pig",
    "pastrami" : "cow",
    "corned beef": "cow",
    "honey ham" : "pig",
    "nova lox" : "salmon"
}

In [ ]:
# Series의 map 메서드: 변형을 위한 매핑 정보가 담긴 딕셔너리 같은 객체나 함수를 인수로 받음
data["animal"] = data["food"].map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,pastrami,6.0,cow
4,corned beef,7.5,cow
5,bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [96]:
# meat_to_animal 반환하는 함수 정의
def get_animal(x):
  return meat_to_animal[x]

# food가 입력되면 각 행에 해당되는 value 반환
data["food"].map(get_animal)

,food
0,pig
1,pig
2,pig
3,cow
4,cow
5,pig
6,cow
7,pig
8,salmon


### 7.2.3 값 치환하기
- replace 메서드

In [97]:
data = pd.Series([1., -999., 2., -999., -1000, 3.])
data # -999는 누락된 데이터를 나타내는 감싯값

,0
0,1.0
1,-999.0
2,2.0
3,-999.0
4,-1000.0
5,3.0


In [98]:
# replace 메서드를 이용해서 감싯값을 판다스에서 인식할 수 있는 NA 값으로 치환해 새로운 Series 생성
data.replace(-999, np.nan)

,0
0,1.0
1,NaN
2,2.0
3,NaN
4,-1000.0
5,3.0


In [99]:
# 여러 개의 값 치환
data.replace([-999, -1000], np.nan)

,0
0,1.0
1,NaN
2,2.0
3,NaN
4,NaN
5,3.0


In [100]:
# 각 값을 다르게 치환
data.replace([-999, -1000], [np.nan, 0])

,0
0,1.0
1,NaN
2,2.0
3,NaN
4,0.0
5,3.0


In [101]:
# 리스트 대신 딕셔너리도 가능
data.replace({-999: np.nan, -1000:0})

,0
0,1.0
1,NaN
2,2.0
3,NaN
4,0.0
5,3.0


### 7.2.4 축 색인 이름 바꾸기
- rename: 원래 객체 변경 없이 새로운 객체 생성
- rename 메서드는 딕셔너리 형식의 객체를 이용해서 축 이름 중 일부만 변경 가능

In [102]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index = ["Ohio", "Colorado", "New York"],
                    columns=["one", "two", "three", "four"]
                    )
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [106]:
# 축 색인과 map 메서드
def transform(x):
  # return x.upper() # 전체 글자 대문자로
  return x[:4].upper() # 앞의 4글자만 대문자로

data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [108]:
# 인덱스를 변화된 값으로 교체
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [109]:
# rename: 원래 객체 변경 없이 새로운 객체 생성
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [110]:
# rename 메서드는 딕셔너리 형식의 객체를 이용해서 축 이름 중 일부만 변경 가능
data.rename(index={"OHIO": "INDIANA"},
            columns={"three": "peekaboo"})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### 7.2.5 이산화
- 데이터 분할 및 구간화 (Discretization and Binning): 연속적인 데이터를 분석을 위해 구간으로 나눔.
- pandas.cut: 지정된 구간(bins)에 따라 데이터를 나눔.
- 결과로 나오는 Categorical 객체는 각 구간을 나타내는 Interval 값 유형을 가짐.
구간 레이블을 사용자 정의할 수 있음.
 - 시적인 구간 경계 대신 정수 개수의 구간을 전달하여 동일 길이의 구간을 만들 수 있음.
- pandas.qcut: 샘플 분위수를 기준으로 데이터를 나눔. 이를 통해 각 구간이 거의 동일한 수의 데이터 포인트를 갖게 됨.


In [125]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

# pandas cut 함수를 이용해 나이별로 그룹 나누기
bins = [18, 25, 35, 60, 100]

"""
간격 표기법: 소괄호와 함께 사용된 값은 미포함
대괄호와 함께 사용된 값은 포함
[코드]
0: 18 - 25
1: 26 - 35
2: 36 - 60
3: 61 - 100
"""

age_categories = pd.cut(ages, bins)
age_categories # 반환된 객체는 Categorical (범주형) 객체이며, 각 그룹은 개별 그룹의 상한과 하한값을 담은 간격으로 구분

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [126]:
age_categories.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [127]:
age_categories.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')

In [124]:
# 구간에 속하지 않을 경우 -1 반환
ages1 = [6, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
bins = [18, 25, 35, 60, 100]

age_categories1 = pd.cut(ages, bins)
print(age_categories1.codes)
print(age_categories1.categories)

[-1  0  0  1  0  0  2  1  3  2  2  1]
IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')


In [114]:
age_categories.categories[0] # 첫 번째 카테고리의 첫 번째 요소

Interval(18, 25, closed='right')

In [115]:
# pandas.cut 결과에 대한 그룹의 개수
pd.value_counts(age_categories)

<ipython-input-115-d4da73026c74>:2: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(age_categories)


,count
"(18, 25]",5
"(25, 35]",3
"(35, 60]",3
"(60, 100]",1


In [132]:
# 간격 표기법: 소괄호와 함께 사용된 값은 미포함 / 대괄호와 함께 사용된 값은 포함
# right=False 전달하여 소괄호와 대괄호의 위치 변경 가능
pd.cut(ages, bins, right=False)

[[18, 25), [18, 25), [25, 35), [25, 35), [18, 25), ..., [25, 35), [60, 100), [35, 60), [35, 60), [25, 35)]
Length: 12
Categories (4, interval[int64, left]): [[18, 25) < [25, 35) < [35, 60) < [60, 100)]

In [133]:
# labels 옵션: 그룹의 이름을 리스트나 배열 형태로 직접 전달
group_names = ["Youth", "YoungAdult", "MiddleAged", "Senior"]
pd.cut(ages, bins, labels=group_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

In [134]:
# pandas.cut에 명시적으로 그룹의 경곗값을 넘기지 않고 그룹의 개수를 넘겨주면
# 데이터의 최솟값과 최댓값을 기준으로 균등한 길이의 그룹을 자동으로 계산
data = np.random.uniform(size=20)
pd.cut(data, 4, precision=2) # 소수점 아래 두 자리까지

[(0.043, 0.25], (0.043, 0.25], (0.66, 0.87], (0.043, 0.25], (0.043, 0.25], ..., (0.66, 0.87], (0.043, 0.25], (0.66, 0.87], (0.46, 0.66], (0.043, 0.25]]
Length: 20
Categories (4, interval[float64, right]): [(0.043, 0.25] < (0.25, 0.46] < (0.46, 0.66] <
                                           (0.66, 0.87]]

In [135]:
# pandas.qcut: 표본 사분위수 기반 데이터 분할
data = np.random.standard_normal(1000)

quartiles = pd.qcut(data, 4, precision=2)
quartiles

[(-0.044, 0.62], (-3.25, -0.71], (0.62, 2.62], (-3.25, -0.71], (-3.25, -0.71], ..., (-3.25, -0.71], (0.62, 2.62], (-0.044, 0.62], (-0.71, -0.044], (-0.044, 0.62]]
Length: 1000
Categories (4, interval[float64, right]): [(-3.25, -0.71] < (-0.71, -0.044] < (-0.044, 0.62] <
                                           (0.62, 2.62]]

In [136]:
pd.value_counts(quartiles)

<ipython-input-136-e680040d00da>:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(quartiles)


,count
"(-3.25, -0.71]",250
"(-0.71, -0.044]",250
"(-0.044, 0.62]",250
"(0.62, 2.62]",250


In [137]:
# pandas.cut 함수처럼 사분위수 직접 지정 가능
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.]).value_counts()

,count
"(-3.237, -1.295]",100
"(-1.295, -0.0445]",400
"(-0.0445, 1.208]",400
"(1.208, 2.624]",100


### 7.2.6 이상치를 찾고 제외하기
- 배열 연산 시는 이상치(outlier)를 제외하거나 적당한 값으로 대체하는 것이 중요

In [138]:
data = pd.DataFrame(np.random.standard_normal((1000,4)))
data

,0,1,2,3
0,1.004671,0.795866,-1.784579,0.386799
1,-0.363529,-0.641460,-0.221975,-0.953347
2,0.167781,1.911261,1.318200,0.842577
3,0.278140,-0.811815,2.465019,-0.360748
4,0.778880,1.446008,0.755362,0.791262
...,...,...,...,...
995,0.722034,-0.505633,-0.976655,0.010074
996,0.744029,0.209542,0.805562,-1.313526
997,-0.020237,0.726995,1.335588,-0.921311
998,1.365158,1.417900,0.662097,-1.715451


In [139]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.052480,0.023049,-0.012388,0.004997
std,1.017415,1.011962,1.022199,0.963586
min,-2.846716,-2.940828,-2.799093,-2.825079
25%,-0.655196,-0.629784,-0.711021,-0.623253
50%,0.079343,0.025114,-0.089080,-0.004612
75%,0.765196,0.702067,0.661583,0.650232
max,4.272766,3.169144,3.830540,4.151425


In [140]:
# DataFrame 열에서 절댓값이 3을 초과하는 값 찾기
col = data[2] # 2열만
col[col.abs() > 3]

,2
208,3.83054


In [141]:
# 절댓값 3을 초과하는 값이 들어있는 모든 행 선택하기(any)
data[(data.abs() > 3).any(axis="columns")] # any: 칼럼 중 하나라도 포함되는 것

,0,1,2,3
99,-0.124584,0.846891,-1.204395,3.160258
117,-1.936182,3.079495,1.435864,0.700223
208,0.937700,1.052159,3.830540,1.012268
297,0.426010,-1.153309,-0.529956,3.079979
591,0.587203,3.169144,-0.502561,-0.930513
709,-0.014751,-0.093413,0.539077,3.358391
770,4.272766,1.863289,-1.281928,0.973352
802,-0.583309,0.436984,-0.355948,4.151425


In [142]:
# np.sign: data가 양수인지 음수인지에 따라 1이나 -1 반환
# 절댓값이 3을 초과하면 양수:1 , 음수:-1 반환하고 3 곱해줌
data[data.abs() > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.051207,0.022800,-0.013219,0.003247
std,1.012917,1.011213,1.019407,0.957348
min,-2.846716,-2.940828,-2.799093,-2.825079
25%,-0.655196,-0.629784,-0.711021,-0.623253
50%,0.079343,0.025114,-0.089080,-0.004612
75%,0.765196,0.702067,0.661583,0.650232
max,3.000000,3.000000,3.000000,3.000000


In [143]:
np.sign(data).head()

,0,1,2,3
0,1.0,1.0,-1.0,1.0
1,-1.0,-1.0,-1.0,-1.0
2,1.0,1.0,1.0,1.0
3,1.0,-1.0,1.0,-1.0
4,1.0,1.0,1.0,1.0


### 7.2.7 뒤섞기와 임의 샘플링
- numoy.random.permutation 함수: Series나 DataFrame의 행을 임의의 순서대로 재배치 (순서를 바꾸고 싶은 만큼의 길이를 전달)

In [144]:
df = pd.DataFrame(np.arange(5 * 7).reshape((5, 7)))
df

,0,1,2,3,4,5,6
0,0,1,2,3,4,5,6
1,7,8,9,10,11,12,13
2,14,15,16,17,18,19,20
3,21,22,23,24,25,26,27
4,28,29,30,31,32,33,34


In [145]:
sampler = np.random.permutation(5)
sampler

array([4, 2, 1, 0, 3])

In [146]:
#  take() 메서드는 입력된 축과 정수 인덱스에 해당하는 요소들을 반환
df.take(sampler)

,0,1,2,3,4,5,6
4,28,29,30,31,32,33,34
2,14,15,16,17,18,19,20
1,7,8,9,10,11,12,13
0,0,1,2,3,4,5,6
3,21,22,23,24,25,26,27


In [147]:
df.iloc[sampler]

,0,1,2,3,4,5,6
4,28,29,30,31,32,33,34
2,14,15,16,17,18,19,20
1,7,8,9,10,11,12,13
0,0,1,2,3,4,5,6
3,21,22,23,24,25,26,27


In [148]:
# take 호출 시 axis="columns" 는 열에 대해 작동
column_sampler = np.random.permutation(7)
column_sampler

array([6, 3, 4, 5, 2, 1, 0])

In [149]:
df.take(column_sampler, axis="columns")

,6,3,4,5,2,1,0
0,6,3,4,5,2,1,0
1,13,10,11,12,9,8,7
2,20,17,18,19,16,15,14
3,27,24,25,26,23,22,21
4,34,31,32,33,30,29,28


In [150]:
# 치환 없이 일부만 선택하려면 Series나 DataFrame의 sample 메서드 사용
df.sample(n=3)

,0,1,2,3,4,5,6
2,14,15,16,17,18,19,20
1,7,8,9,10,11,12,13
0,0,1,2,3,4,5,6


In [151]:
# 반복 선택을 허용하기 위해 replace=True 옵션 전달
choices = pd.Series([5, 7, -1, 6, 4])
choices.sample(n=10, replace=True) # 중복을 허용하며 10개 생성(인덱스 번호도 출력됨)

,0
1,7
4,4
0,5
1,7
2,-1
2,-1
3,6
2,-1
2,-1
4,4


### 7.2.8 표시자, 더미 변수 계산하기
- 통계 모델이나 머신러닝을 위한 데이터 변환: 분륫값을 더미나 표시자 행렬로 전환
* 더미데이터: 유용한 데이터가 없지만 공간을 예비해두어 실제 데이터가 존재하는 것처럼 다루는 데이터
- ex) 값으로 1과 0을 채워넣는 것
- pandas.get_dummies 함수 사용: 범주형 데이터를 원-핫 인코딩된 DataFrame으로 변환
- 표시자 DataFrame 열에 접두어(perfix)를 추가한 후 다른 데이터와 병합하고 싶은 경우, pandas.get_dummies 함수의 prefix 인수 사용
- 여러 범주에 속하는 행의 경우, str.get_dummies 메서드를 사용하여 구분된 문자열에서 더미 변수 생성 가능
- pandas.cut과 함께 pandas.get_dummies 사용하여 데이터 구간을 더미 변수로 변환

In [152]:
df = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"],
                   "data1": range(6)
                   })
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [153]:
pd.get_dummies(df["key"], dtype=float)

,a,b,c
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,0.0,0.0,1.0
4,1.0,0.0,0.0
5,0.0,1.0,0.0


In [155]:
pd.get_dummies(df["key"], prefix="key", dtype=float)

,key_a,key_b,key_c
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,0.0,0.0,1.0
4,1.0,0.0,0.0
5,0.0,1.0,0.0


In [154]:
# 표시자 DataFrame 열에 접두어(perfix)를 추가한 후 다른 데이터와 병합하고 싶은 경우
# pandas.get_dummies 함수의 prefix 인수 사용
dummies = pd.get_dummies(df["key"], prefix="key", dtype=float)
df_with_dummy = df[["data1"]].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0.0,1.0,0.0
1,1,0.0,1.0,0.0
2,2,1.0,0.0,0.0
3,3,0.0,0.0,1.0
4,4,1.0,0.0,0.0
5,5,0.0,1.0,0.0


#### DataFrame의 한 행이 여러 범주에 속할 경우 다른 방식 사용
- str.get_dumiies 메서드를 사용하여 구분된 문자열에서 더미 변수 생성 가능

In [156]:
mnames = ["movie_id", "title", "genres"]
movies = pd.read_table(path + '/datasets/movielens/movies.dat', sep="::",
                       header=None, names=mnames, engine="python")
movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [157]:
dummies = movies["genres"].str.get_dummies("|")
dummies.iloc[:10, :6] # 9번행, 5번열까지 불러오기

,Action,Adventure,Animation,Children's,Comedy,Crime
0,0,0,1,1,1,0
1,0,1,0,1,0,0
2,0,0,0,0,1,0
3,0,0,0,0,1,0
4,0,0,0,0,1,0
5,1,0,0,0,0,1
6,0,0,0,0,1,0
7,0,1,0,1,0,0
8,1,0,0,0,0,0
9,1,1,0,0,0,0


In [159]:
# dummies와 movies를 결합하고 add_prefix 메서드를 이용해 dummies의 열 이름에 "Genre_" 추가
movies_windic = movies.join(dummies.add_prefix("Genre_"))
# movies_windic.iloc[0]
# movies_windic.iloc[0:2]
movies_windic.iloc[0:2, :6]

,movie_id,title,genres,Genre_Action,Genre_Adventure,Genre_Animation
0,1,Toy Story (1995),Animation|Children's|Comedy,0,0,1
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0,1,0


#### pandas.get_dummies 와 이산함수 (pandas.cut 등)의 조합

In [160]:
np.random.seed(12345) # 난수 반복 가능하도록 시드 값 고정
values = np.random.uniform(size=10)
values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [161]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1] # (0.0, 0.2]: 0.0 초과 0.2 이하
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,False,False,False,False,True
1,False,True,False,False,False
2,True,False,False,False,False
3,False,True,False,False,False
4,False,False,True,False,False
5,False,False,True,False,False
6,False,False,False,False,True
7,False,False,False,True,False
8,False,False,False,True,False
9,False,False,False,True,False


## 7.3. 확장 데이터 유형
[(넘파이 기반 구축으로 인한) 판다스의 단점]
- 정수나 불리언형을 가진 누락된 데이터 처리의 불완전: 결측치가 생성되는 경우 판다스는 자료형을 float64로 변경하고 np.nan을 이용해서 null 값 표현 > 판다스 알고리즘에 미묘한 이슈 유발 및 복합적인 영향
- 대량의 문자열 데이터셋의 계산 비용이 높고 메모리를 많이 사용
- 시간 간격이나 시간 차이, 시간대가 포함된 타임스탬프 같은 일부 자료형에서 계산 비용이 높은 파이썬 객체를 사용하지 않고는 효과적으로 처리하는 것이 불가능
<br> [판다스 확장 데이터 유형]
- BooleanDtype: 널 값이 가능한 불리언 데이터. "boolean"
- CategoricalDtype: 범주형 데이터. "category"
- DatetimeTZDtype: 시간대가 포함된 Datatime
- Float32Dtype: 널 값이 가능한 32비트 부동소수형. "Float32"
- Float64Dtype: 널 값이 가능한 64비트 부동소수형. "Float64"
- Int8Dtype: 널 값이 가능한 부호가 있는 8비트 정수형. "Int8"
- Int16Dtype: 널 값이 가능한 부호가 있는 16비트 정수형. "Int16"
- Int32Dtype: 널 값이 가능한 부호가 있는 32비트 정수형. "Int32"
- Int64Dtype: 널 값이 가능한 부호가 있는 64비트 정수형. "Int64"
- UInt8Dtype: 널 값이 가능한 부호가 없는 8비트 정수형. "UInt8"
- UInt16Dtype: 널 값이 가능한 부호가 없는 16비트 정수형. "UInt16"
- UInt32Dtype: 널 값이 가능한 부호가 없는 32비트 정수형. "UInt32"
- UInt64Dtype: 널 값이 가능한 부호가 없는 64비트 정수형. "UInt64"

In [ ]:
s = pd.Series([1, 2, 3, None])
s

,0
0,1.0
1,2.0
2,3.0
3,NaN


In [ ]:
s.dtype

dtype('float64')

In [ ]:
s = pd.Series([1, 2, 3, None], dtype=pd.Int64Dtype)
s

<ipython-input-99-8e20a6178654>:1: UserWarning: Instantiating Int64Dtype without any arguments.Pass a Int64Dtype instance to silence this warning.
  s = pd.Series([1, 2, 3, None], dtype=pd.Int64Dtype)


,0
0,1
1,2
2,3
3,<NA>


In [ ]:
s.isna()

,0
0,False
1,False
2,False
3,True


In [ ]:
s.dtype

Int64Dtype()

In [ ]:
print(s[3])
print(s[3] is pd.NA)

<NA>
True


In [ ]:
s = pd.Series([1, 2, 3, None], dtype="Int64")
s

,0
0,1
1,2
2,3
3,<NA>


In [ ]:
s = pd.Series(["one", "two", None, "three"], dtype=pd.StringDtype())
s

,0
0,one
1,two
2,<NA>
3,three


In [ ]:
# Series의 astype 메서드에 확장형을 인수로 전달하면 변환 가능
df = pd.DataFrame({"A": [1, 2, None, 4],
                   "B": ["one", "two", "three", None],
                   "C": [False, None, False, True]
                  })
df

,A,B,C
0,1.0,one,False
1,2.0,two,None
2,NaN,three,False
3,4.0,None,True


In [ ]:
df["A"] = df["A"].astype("Int64")

df["B"] = df["B"].astype("string")

df["C"] = df["C"].astype("boolean")

df

,A,B,C
0,1,one,False
1,2,two,<NA>
2,<NA>,three,False
3,4,<NA>,True


## 7.4 문자열 다루기

### 7.4.1 파이썬 내장 문자열 객체 메서드
- count: 문자열에서 겹치지 않는 부분 문자열의 개수 반환
- endswith: 문자열이 주어진 접미사로 끝날 경우 True
- startswith: 문자열이 주어진 접두사로 시작할 경우 True
- join: 문자열을 구분자로 하고 다른 문자열을 순서대로 이어 붙임
- index: 부분 문자열의 첫 번째 글자의 위치 반환. 없을 경우 ValueError
- find: 첫 번째 부분 문자열의 첫 번째 글자의 위치 반환. index와 유사하나, 없을 경우 -1 반환
- rfind: 마지막 부분 문자열의 첫 번째 글자의 위치 반환. 없을 경우 -1 반환
- replace: 문자열을 다른 문자열로 치환
- strip, rstrip, lstrip: 개행 문자를 포함한 공백 문자 제거 (rstrip: 문자열의 마지막 부분의 공백 문자 제거 / lstrip: 문자열의 시작 부분)
- split: 구분자를 기준으로 문자열을 부분 문자열의 리스트로 분리
- lower: 알파벳 문자를 소문자로 변환
- upper: 알파멧 문자를 대문자로 변환
- casefold: 문자를 소문자로 변환. 지역 문자들은 그에 상응하는 대체 문자로 교체
- ljust, rjust: 문자열을 오른쪽 또는 왼쪽으로 정렬하고 주어진 길이에서 문자열의 길이를 제외한 나머지 부분은 공백 문자 채워넣음

In [ ]:
val = "a,b, guido"
val.split(",")

['a', 'b', ' guido']

In [ ]:
pieces = [x.strip() for x in val.split(",")]
pieces

['a', 'b', 'guido']

In [ ]:
first, second, third = pieces

first + "::" + second + "::" + third

'a::b::guido'

In [ ]:
"::".join(pieces)

'a::b::guido'

In [ ]:
# 일치하는 부분 문자열 찾기 (in)
"guido" in val

True

In [ ]:
val.index(",")

1

In [ ]:
val.find(":")

-1

In [ ]:
# count: 해당 문자열 개수
val.count(",")

2

In [ ]:
# replace: 치환
val.replace(",", "::")

'a::b:: guido'

In [ ]:
val.replace(",", " ")

'a b  guido'

### 7.4.2 정규 표현식 (regex)
- re 모듈
- findall: 문자열에서 겹치지 않는 모든 발견된 패턴을 리스트로 반환
- finditer: findall과 같으나 발견된 패턴을 이터레이터를 통해 하나씩 반환
- match: 문자열의 시작점부터 패턴을 찾고 선택적으로 패턴 컴포넌트를 그룹으로 나눔. 일치하는 패턴이 있을 경우 match 객체 반환, 없을 경우 None
- search: 문자열에서 패턴과 일치하는 내용을 검색하고 match 객체 반환. match 메서드와는 다르게 시작부터 일치하는 내용만 찾지 않고 문자열 어디든 일치하는 내용이 있다면 반환
- split: 문자열에서 패턴과 일치하는 부분을 분리
- sub.subn: 문자열에서 일치하는 모든 패턴(sub) 혹은 처음 n개의 패턴(subn)을 대체 표현으로 치환. 대체 표현 문자열은 \1, \2 와 같은 기호를 사용해서 매치 글부의 요소 참조

In [ ]:
import re

In [ ]:
text = "foo   bar\t baz \tqux"
re.split(r"\s+", text) # 정규 표현식이 컴파일(번역) 되고 split 메서드 실행

['foo', 'bar', 'baz', 'qux']

In [ ]:
# re.compile을 통해 직접 정규 표현식 컴파일 가능
# 동일한 정규 표현식을 다른 문자열에 적용해야 할 경우 re.compile을 이용하여 정규 표현식 객체 생성
regex = re.compile(r"\s+")
regex.split(text)

['foo', 'bar', 'baz', 'qux']

In [ ]:
# findall: 정규 표현식에 매칭되는 모든 패턴의 목록 얻기
regex.findall(text)

['   ', '\t ', ' \t']

In [ ]:
# search: 패턴과 일치하는 첫 번째 항목 반환
# match: 문자열의 시작 부분에서 일치하는 것만 찾음
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com"""

pattern = r"[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}"

# re.IGNORECASE는 정규 표현식이 대소문자를 구분하지 않도록 함
regex = re.compile(pattern, flags=re.IGNORECASE)

# findall 메서드를 사용하여 이메일 주소 리스트 생성
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

In [ ]:
# search는 첫 번째 이메일 주소만 찾음
m = regex.search(text)
m

<re.Match object; span=(5, 20), match='dave@google.com'>

In [ ]:
text[m.start():m.end()]

'dave@google.com'

In [ ]:
# regex.match: 문자열의 시작점에서부터 패턴이 일치하는지 검사
print(regex.match(text))

None


In [ ]:
# sub: 찾은 패턴을 주어진 문자열로 치환하고 새로운 문자열로 반환
print(regex.sub("REDACTED", text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED


In [ ]:
# 이메일 주소 찾기 + 각 주소를 사용자 이름, 도메인 이름, 도메인 접미사로 나누기
pattern = r"([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Za-z]{2,4})" # 각 패턴을 괄호로 묶어주면 됨
regex = re.compile(pattern, flags=re.IGNORECASE)

In [ ]:
# match 객체를 이용하면 groups 메서드로 각 패턴 컴포넌트의 튜플 얻음
m = regex.match("jun@naver.com")

m.groups()

('jun', 'naver', 'com')

In [ ]:
# 패턴에 그룹이 존재할 경우 findall 메서드는 튜플의 목록 반환
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

In [ ]:
# sub는 \1, \2 같은 특수한 기호를 사용해서 각 패턴 그룹에 접근 가능
# \1: 첫번째로 찾은 그룹, \2: 두번째로 찾은 그룹
print(regex.sub(r"Username: \1, Domain: \2, Suffix: \3", text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com


### 7.4.3 판다스의 문자열 함수
- cat: 선택적인 구분자와 함께 요소별로 문자열 이어 붙임
- contains: 문자열이 패턴이나 정규 표현식을 포함하는지를 나타내는 불리언 배열 반환
- count: 일치하는 패턴 수 반환
- extract: 문자열이 담긴 Series에서 하나 이상의 문자열을 추출하기 위해 정규 표현식을 이용. 결과는 각 그룹이 하나의 열이 되는 DataFrame
- endswith: 각 요소에 대해 x.endswith(pattern)과 동일하게 작동
- startswith: 각 요소에 대해 x.startswith(pattern)과 동일하게 작동
- findall: 각 문자열에 대해 일치하는 패턴/정규 표현식의 전체 목록 구함
- get: i번째 요소 반환
- isalnum: 내장 함수 str.isalnum과 동일
- isalpha: 내장 함수 str.isalpha와 동일
- isdecimal: 내장 함수 str.isdecimal과 동일
- isdigit: 내장 함수 str.isdigit과 동일
- islower: 내장 함수 str.islower와 동일
- is.numeric; 내장 함수 str.isnumeric과 동일
- isupper: 내장 함수 str.isupper와 동일
- join: Series의 각 요소를 주어진 구분자로 연결
- len: 각 문자열의 길이 구함
- lower, upper: 대소문자로 변환.
- match: 주어진 정규 표현식으로 각 요소에 대한 re.match 수행하여 일치 여부를 불리언 타입으로 반환
- pad: 문자열의 좌우 혹은 양쪽에 공백 추가
- center: pad(side="both")와 동일
- repeat: 값 복사
- replace: 패턴/정규 표현식과 일치하는 내용을 다른 문자열로 치환
- slice: Series 안에 있는 각 문자열 자름
- split: 정규 표현식 혹은 구분자로 문자열 나눔
- strip: 개행 문자를 포함해 왼쪽과 오른쪽 공백 문자 제거
- rstrip: 오른쪽 / lstrip: 왼쪽

In [ ]:
data = {"Dave": "dave@google.com", "Steve": "steve@gmail.com",
        "Rob": "rob@gmail.com", "Wes": "np.nan"}
data = pd.Series(data)
data

,0
Dave,dave@google.com
Steve,steve@gmail.com
Rob,rob@gmail.com
Wes,np.nan


In [ ]:
data.isna()

,0
Dave,False
Steve,False
Rob,False
Wes,False


In [ ]:
# str.contains: 포함 여부 검사 - 불리언 값 반환
data.str.contains("gmail")

,0
Dave,False
Steve,True
Rob,True
Wes,False


In [ ]:
data_as_string_ext = data.astype('string')
data_as_string_ext

,0
Dave,dave@google.com
Steve,steve@gmail.com
Rob,rob@gmail.com
Wes,np.nan


In [ ]:
data_as_string_ext.str.contains("gmail")

,0
Dave,False
Steve,True
Rob,True
Wes,False


In [ ]:
pattern = r"([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Za-z]{2,4})"

data.str.findall(pattern, flags=re.IGNORECASE)

,0
Dave,"[(dave, google, com)]"
Steve,"[(steve, gmail, com)]"
Rob,"[(rob, gmail, com)]"
Wes,[]


In [ ]:
# 벡터화된 요소 꺼내기
matches = data.str.findall(pattern, flags=re.IGNORECASE).str[0]
matches

,0
Dave,"(dave, google, com)"
Steve,"(steve, gmail, com)"
Rob,"(rob, gmail, com)"
Wes,NaN


In [ ]:
matches.str.get(1)

,0
Dave,google
Steve,gmail
Rob,gmail
Wes,NaN


In [ ]:
# 문자열 잘라내기
data.str[:5]

,0
Dave,dave@
Steve,steve
Rob,rob@g
Wes,np.na


In [ ]:
# str.extract: 정규 표현식의 결과를 DataFrame 형태로 반환
data.str.extract(pattern, flags=re.IGNORECASE)

,0,1,2
Dave,dave,google,com
Steve,steve,gmail,com
Rob,rob,gmail,com
Wes,NaN,NaN,NaN


## 7.5 범주형 데이터

### 7.5.1 개발 배경과 동기
- 데이터 웨어하우스, 통계 컴퓨팅 등 다양한 데이터 시스템에서 범주형 표현을 사용하면 분석 작업에서 성능 향상을 얻을 수 있음

In [162]:
values = pd.Series(['apple', 'orange', 'apple', 'apple'] * 2)
values

,0
0,apple
1,orange
2,apple
3,apple
4,apple
5,orange
6,apple
7,apple


In [163]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [164]:
# 각 value의 빈도수 계산
pd.value_counts(values)

<ipython-input-164-e9d61bad5d62>:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(values)


,count
apple,6
orange,2


In [165]:
values = pd.Series([0, 1, 0, 0] * 2)
values

,0
0,0
1,1
2,0
3,0
4,0
5,1
6,0
7,0


In [166]:
dim = pd.Series(['apple', 'oirange'])
dim

,0
0,apple
1,oirange


In [167]:
# take 메서드 사용하여 Series에 저장된 문자열 구하기
dim.take(values)

,0
0,apple
1,oirange
0,apple
0,apple
0,apple
1,oirange
0,apple
0,apple


### 7.5.2. 판다스의 Categorical 확장형
- Categorical 확장형: 정수 기반의 범주형 데이터를 표현하는 특수한 데이터 유형
- 주로 문자열 데이터에서 유사한 값이 다수 존재하는 경우 데이터를 효과적으로 압축해서 적은 메모리에서도 빠른 성능을 냄
- 범주형 배열은 변경이 불가능한 값이라면 어떤 자료형이라도 포함 가능


In [168]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2
N = len(fruits) # 8개
rng = np.random.default_rng(seed=12345)

df = pd.DataFrame({"fruit": fruits,
                   "basket_id": np.arange(N),
                   "count": rng.integers(3, 15, size=N), # 3과 15 사이의 값 N개
                   "weight": rng.uniform(0, 4, size=N)}, # 0과 4 사이의 값 N개
                   columns = ["basket_id", "fruit", "count", "weight"]
                  )
df

,basket_id,fruit,count,weight
0,0,apple,11,1.564438
1,1,orange,5,1.331256
2,2,apple,12,2.393235
3,3,apple,6,0.746937
4,4,apple,5,2.691024
5,5,orange,12,3.767211
6,6,apple,10,0.992983
7,7,apple,11,3.795525


In [169]:
# df['fruit'] 범주형 데이터로 변경하기
# astype: 자료형 변경
fruit_cat = df['fruit'].astype('category')
fruit_cat

,fruit
0,apple
1,orange
2,apple
3,apple
4,apple
5,orange
6,apple
7,apple


In [170]:
# .array 속성으로 접근하기
c = fruit_cat.array
type(c)

pandas.core.arrays.categorical.Categorical

In [171]:
# Categorical 객체
c.categories

Index(['apple', 'orange'], dtype='object')

In [172]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

In [173]:
# categories와 codes 속성 간의 매핑 가져오기
dict(enumerate(c.categories))

{0: 'apple', 1: 'orange'}

In [174]:
# 변환 완료된 값 대입하여 DataFrame의 열을 범주형으로 변경하기
df['fruit'] = df['fruit'].astype('category')
df['fruit']

,fruit
0,apple
1,orange
2,apple
3,apple
4,apple
5,orange
6,apple
7,apple


In [ ]:
# 파이썬 시퀀스에서 pd.Categorical 직접 생성하기
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])
my_categories

['foo', 'bar', 'baz', 'foo', 'bar']
Categories (3, object): ['bar', 'baz', 'foo']

In [ ]:
# from_codes로 범주형 데이터 생성하기 (기존에 정의된 범주와 범주 코드 있을 경우)
categories = ['foo', 'bar', 'baz']
codes = [0, 1, 2, 0, 0, 1]

my_cats_2 = pd.Categorical.from_codes(codes, categories)
my_cats_2

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo', 'bar', 'baz']

In [ ]:
# 범주형으로 변경 시 특정 순서 보장 하지 않음.
# 범주형 배열은 입력 데이터의 순서에 따라 순서 다를 수 있으므로
# from_codes를 사용하거나 다른 범주형 데이터 생성자 이용하여 순서 지정
ordered_cat = pd.Categorical.from_codes(codes, categories,
                                        ordered=True)
ordered_cat

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

In [ ]:
# as_ordered: 순서가 없는 범주형 인스턴스 정렬
my_cats_2.as_ordered()

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

### 7.5.3 Categorical 연산

In [ ]:
rng = np.random.default_rng(seed=12345)

draws = rng.standard_normal(1000)
draws[:5]

array([-1.42382504,  1.26372846, -0.87066174, -0.25917323, -0.07534331])

In [ ]:
# 데이터를 사분위수로 나누고 통계 내기
bins = pd.qcut(draws, 4)
bins

[(-3.121, -0.675], (0.687, 3.211], (-3.121, -0.675], (-0.675, 0.0134], (-0.675, 0.0134], ..., (0.0134, 0.687], (0.0134, 0.687], (-0.675, 0.0134], (0.0134, 0.687], (-0.675, 0.0134]]
Length: 1000
Categories (4, interval[float64, right]): [(-3.121, -0.675] < (-0.675, 0.0134] < (0.0134, 0.687] <
                                           (0.687, 3.211]]

In [ ]:
# qcut 함수의 labels 인수를 통해 직접 이름 짓기
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
bins

['Q1', 'Q4', 'Q1', 'Q2', 'Q2', ..., 'Q3', 'Q3', 'Q2', 'Q3', 'Q2']
Length: 1000
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [ ]:
bins.codes[:10]

array([0, 3, 0, 1, 1, 0, 0, 2, 2, 0], dtype=int8)

In [ ]:
# groupby를 이용한 요약 통계 추출
bins = pd.Series(bins, name='quartile')

results = (pd.Series(draws)
          .groupby(bins)
          .agg(['count', 'max', 'min'])
          .reset_index()
          )

results

<ipython-input-179-60ad9f8aa3f0>:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(bins)


,quartile,count,max,min
0,Q1,250,-0.678494,-3.119609
1,Q2,250,0.008009,-0.673305
2,Q3,250,0.686183,0.018753
3,Q4,250,3.211418,0.688282


In [ ]:
# quartile 열은 bins 순서를 포함한 원래 범주 정보 유지
results['quartile']

,quartile
0,Q1
1,Q2
2,Q3
3,Q4


#### 범주형을 활용한 성능 개선

In [ ]:
# 소수의 독립적인 범주로 분류되는 1천만 개의 값을 가진 Series 생성
N = 10_000_000

labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N // 4))

# labels를 범주형으로 변환
categories = labels.astype('category')

# labels와 categories 메모리 사용량 비교
print(f"labels 메모리 사용량: {labels.memory_usage(deep=True)}")
print(f"categories 메모리 사용량: {categories.memory_usage(deep=True)}")

labels 메모리 사용량: 600000132
categories 메모리 사용량: 10000544


In [ ]:
# 변환 시간 체크
%time _ = labels.astype('category')

CPU times: user 507 ms, sys: 210 ms, total: 717 ms
Wall time: 710 ms


In [ ]:
# labels와 categories의 value_counts() 함수 성능 비교
%timeit labels.value_counts()

499 ms ± 9.36 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%timeit categories.value_counts()

72.4 ms ± 12 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### 7.5.4 Categorical 메서드
- add_categories: 기존 카테고리 끝에 새로운 카테고리 추가
- as_ordered: 카테고리가 순서를 갖도록 함
- as_unordered: 카테고리가 순서를 갖지 않도록 함
- remove_categories: 카테고리 제거. 해당 카테고리에 속한 값들은 null 로 설정
- remove_unused_categories: 데이터에서 관측되지 않는 카테고리 삭제
- renmae_categories: 카테고리 이름을 지정한 이름으로 변경. 카테고리 수 변하지 않음.
- reorder_categories; rename_categories 와 유사하나, 새로운 카테고리가 순서 가짐
- set_categories: 지정한 새로운 카테고리로 카테고리 변경. 카테고리 추가나 삭제 가능

In [ ]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)
cat_s = s.astype('category')
cat_s

,0
0,a
1,b
2,c
3,d
4,a
5,b
6,c
7,d


In [ ]:
# cat을 통해 Categorical 메서드에 접근하기
cat_s.cat.codes

,0
0,0
1,1
2,2
3,3
4,0
5,1
6,2
7,3


In [ ]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

In [ ]:
# set_categories 메서드를 이용해 카테고리 변경하기
actual_categories = ['a', 'b', 'c', 'd', 'e']

cat_s2 = cat_s.cat.set_categories(actual_categories)
cat_s2

,0
0,a
1,b
2,c
3,d
4,a
5,b
6,c
7,d


In [ ]:
cat_s.value_counts()

,count
a,2
b,2
c,2
d,2


In [ ]:
cat_s2.value_counts()

,count
a,2
b,2
c,2
d,2
e,0


In [ ]:
# remove_unused_categories로 관측되지 않는 범주 제거하기
cat_s3 = cat_s[cat_s.isin(['a', 'b'])]
cat_s3

,0
0,a
1,b
4,a
5,b


In [ ]:
cat_s3.cat.remove_unused_categories()

,0
0,a
1,b
4,a
5,b


#### 모델링을 위한 더미변수 생성하기

In [ ]:
cat_s = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype='category')
cat_s

,0
0,a
1,b
2,c
3,d
4,a
5,b
6,c
7,d


In [ ]:
# pd.get_dummies 함수는 1차원 범주형 데이터를 더미 변수를 가진 DataFrame으로 변환
pd.get_dummies(cat_s, dtype=float)

,a,b,c,d
0,1.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0
2,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0
5,0.0,1.0,0.0,0.0
6,0.0,0.0,1.0,0.0
7,0.0,0.0,0.0,1.0
